# Narrative Space Builder (Game Traits)

Use this notebook to build and inspect vectors strictly from game trait names.


In [ ]:
import sys
from pathlib import Path

for base in [Path.cwd(), Path.cwd().parent]:
    if (base / 'src' / 'dungeonbreak_narrative').exists():
        sys.path.insert(0, str(base))
        break

from dungeonbreak_narrative import (
    distance,
    load_game_traits_manifest,
    load_narrative_snapshot,
    trait_map_from_vector,
    validate_game_alignment_warn_only,
    vector_from_trait_map,
)

import numpy as np


In [ ]:
alignment = validate_game_alignment_warn_only()
for warning in alignment['warnings']:
    print('WARN:', warning)

traits = load_game_traits_manifest()
snapshot = load_narrative_snapshot()
print('Traits:', traits)


## Build a Trait Vector from Named Components


In [ ]:
# Only trait names from the manifest are valid keys here.
example_trait_map = {
    'Direction': 0.75,
    'Empathy': 0.35,
    'Survival': 0.60,
}

v = vector_from_trait_map(example_trait_map, trait_names=traits)
print(v)
print(trait_map_from_vector(v, trait_names=traits))


entities = snapshot.get('entities', [])
rows = []
for entity in entities:
    name = entity.get('name', '')
    start_map = entity.get('starting_coordinates') or {}
    if not start_map:
        continue
    rows.append((name, vector_from_trait_map(start_map, trait_names=traits)))

if len(rows) < 2:
    print('Data unavailable: need at least two entities with starting_coordinates in narrative_snapshot.json')
else:
    names = [name for name, _ in rows]
    mat = np.zeros((len(rows), len(rows)), dtype=float)
    for i, (_, va) in enumerate(rows):
        for j, (_, vb) in enumerate(rows):
            mat[i, j] = distance(va, vb)
    for i, name in enumerate(names):
        values = ', '.join([f'{mat[i, j]:.3f}' for j in range(len(names))])
        print(f'{name}: {values}')


In [ ]:
entities = snapshot.get('entities', [])
rows = []
for entity in entities:
    name = entity.get('name', '')
    start_map = entity.get('starting_coordinates') or {}
    if not start_map:
        continue
    rows.append((name, vector_from_trait_map(start_map, trait_names=traits)))

if len(rows) < 2:
    print('Data unavailable: need at least two entities with starting_coordinates in narrative_snapshot.json')
else:
    names = [name for name, _ in rows]
    mat = np.zeros((len(rows), len(rows)), dtype=float)
    for i, (_, va) in enumerate(rows):
        for j, (_, vb) in enumerate(rows):
            mat[i, j] = distance(va, vb)
    print(pd.DataFrame(mat, index=names, columns=names))


dialog_rows = []
probe = vector_from_trait_map({'Direction': 0.5, 'Empathy': 0.5}, trait_names=traits)
for dialog in snapshot.get('dialogs', []):
    location_map = dialog.get('location') or {}
    if not location_map:
        continue
    dv = vector_from_trait_map(location_map, trait_names=traits)
    dialog_rows.append((dialog.get('label', ''), distance(probe, dv)))

if not dialog_rows:
    print('Data unavailable: no dialog location vectors found in snapshot.')
else:
    dialog_rows.sort(key=lambda x: x[1])
    for label, dist in dialog_rows:
        print({'label': label, 'distance_to_probe': round(dist, 6)})


In [ ]:
dialog_rows = []
probe = vector_from_trait_map({'Direction': 0.5, 'Empathy': 0.5}, trait_names=traits)
for dialog in snapshot.get('dialogs', []):
    location_map = dialog.get('location') or {}
    if not location_map:
        continue
    dv = vector_from_trait_map(location_map, trait_names=traits)
    dialog_rows.append((dialog.get('label', ''), distance(probe, dv)))

if not dialog_rows:
    print('Data unavailable: no dialog location vectors found in snapshot.')
else:
    print(pd.DataFrame(dialog_rows, columns=['label', 'distance_to_probe']).sort_values('distance_to_probe'))
